In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import numpy as np
import shared
import json


ve_or_ju = "ju"

In [2]:
history = pd.read_csv(f'data/grouped_paces_{ve_or_ju}.tsv', delimiter="\t")
history["first_name"] = history.name.str.split(" ", expand=True).iloc[:, 0]
history.head()

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,most_common_leg,most_common_country,pace_1,pace_2,pace_3,pace_4,pace_5,pace_6,pace_7,first_name
0,18.5,KOOVEE,joni hirvikallio,6,5,5.905,0.177,1,FIN,5.873,5.762,6.251,5.843,5.796,NaN,NaN,joni
1,12.0,KOOVEE,topi anjala,7,7,6.051,0.483,1,FIN,5.882,5.621,6.100,5.977,7.164,5.642,5.974,topi
2,3.0,DELTA;KOOVEE;HIIDENKIERTÄJÄT,lauri sild,7,7,5.933,0.382,2,FIN,5.836,5.851,5.996,5.829,6.765,5.391,5.866,lauri
3,18.0,OK ORION;KOOVEE,oleksandr kratov,3,3,6.069,0.363,4,SWE,5.703,5.940,6.563,NaN,NaN,NaN,NaN,oleksandr
4,21.0,KANGASALA SK;KOOVEE,kenny kivikas,7,5,5.936,0.345,7,FIN,5.603,5.662,5.837,6.014,6.562,NaN,NaN,kenny


In [3]:
with_history = history[history["num_valid_times"] >= 1]
names = with_history[['first_name', "mean_pace"]].groupby('first_name').agg(["median", "mean", "std"]).dropna()
names.columns = ['_'.join(col).strip() for col in names.columns.values]
names.head()

,mean_pace_median,mean_pace_mean,mean_pace_std
first_name,,,
,9.875,10.154242,3.138480
aapeli,10.896,10.896000,1.623517
aapo,10.355,10.327200,2.310248
aare,11.129,11.129000,1.606547
aarne,11.999,12.280571,2.515633


In [4]:
names.shape

(1587, 3)

In [5]:
X = names[["mean_pace_median", "mean_pace_mean"]].values
pace_kmeans = KMeans(n_clusters=10, random_state=2019).fit(X)

In [6]:
X_std = names[["mean_pace_std"]].values
pace_std_kmeans = KMeans(n_clusters=4, random_state=2019).fit(X_std)

In [7]:
names = names.assign(fn_pace_class=pace_kmeans.labels_)
names = names.assign(fn_pace_std_class=pace_std_kmeans.labels_)
names.head(10)

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
,9.8750,10.154242,3.138480,3,3
aapeli,10.8960,10.896000,1.623517,6,0
aapo,10.3550,10.327200,2.310248,6,3
aare,11.1290,11.129000,1.606547,8,0
aarne,11.9990,12.280571,2.515633,5,3
aarni,13.6145,12.281000,3.609095,5,1
aarno,11.9900,13.449000,3.592685,5,1
aaro,9.3370,9.983316,2.839156,3,3
aaron,11.7250,12.727800,3.386614,5,3


In [8]:
names.sort_values(by=["mean_pace_median"])

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
frédéric,5.4320,5.432000,0.357796,9,0
milos,6.0015,6.001500,0.300520,9,0
gaute,6.1005,6.281000,0.619428,9,0
joacim,6.1340,7.057667,1.688062,9,0
kiril,6.2100,7.240333,1.809763,9,3
westergård,6.2150,6.215000,0.380423,9,0
gernot,6.2180,7.257333,2.216943,9,3
riccardo,6.3395,6.339500,0.559321,9,0
evgeni,6.3950,6.896667,0.919599,9,0


In [9]:
names.columns = names.columns.get_level_values(0)
#names['first_name'] = names.index
names

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
,9.8750,10.154242,3.138480,3,3
aapeli,10.8960,10.896000,1.623517,6,0
aapo,10.3550,10.327200,2.310248,6,3
aare,11.1290,11.129000,1.606547,8,0
aarne,11.9990,12.280571,2.515633,5,3
aarni,13.6145,12.281000,3.609095,5,1
aarno,11.9900,13.449000,3.592685,5,1
aaro,9.3370,9.983316,2.839156,3,3
aaron,11.7250,12.727800,3.386614,5,3


In [10]:
names[["fn_pace_class", "fn_pace_std_class"]].to_csv(f"data/name_pace_classes_{ve_or_ju}.tsv", "\t")